In [ ]:
#| hide
from nbstata.helpers import *
from nbstata.parsers import *

# nbstata

> Jupyter kernel for Stata based on pystata

## Install

```sh
pip install nbstata
```

## How to use

Under construction

## Contributing

`nbstata` is being developed using [nbdev](https://nbdev.fast.ai/). The "nbs" directory is where edits to the source code should be made. (The python code is then exported to the "nbdev" library folder.) The one exception is `install.py`.

The [@patch_to](https://fastcore.fast.ai/basics.html#patch_to) decorator is occasionally used to break up class definitions into separate cells.

For more, see [CONTRIBUTING.md](https://github.com/hugetim/nbstata/blob/master/CONTRIBUTING.md).

## Acknowledgements

Kyle Barron authored the original `stata_kernel` and Vinci Chow carried that work forward for Stata 17, converting the backend to use [pystata](https://www.stata.com/python/pystata/). `nbstata` is directly derived from his [pystata-kernel](https://github.com/ticoneva/pystata-kernel).